In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [30]:
df = pd.read_csv('yellow_tripdata_2021-03.csv') # Load tha dataset
df

C:\Users\ASUS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2021-03-01 00:22:02,2021-03-01 00:23:22,1.0,0.00,1.0,N,264,264,2.0,3.00,0.50,0.5,0.00,0.00,0.3,4.30,0.0
1,2.0,2021-03-01 00:24:48,2021-03-01 00:24:56,1.0,0.00,1.0,N,152,152,2.0,2.50,0.50,0.5,0.00,0.00,0.3,3.80,0.0
2,2.0,2021-03-01 00:25:17,2021-03-01 00:31:01,1.0,0.00,1.0,N,152,152,2.0,3.50,0.50,0.5,0.00,0.00,0.3,4.80,0.0
3,1.0,2021-03-01 00:07:40,2021-03-01 00:31:23,0.0,16.50,4.0,N,138,265,1.0,51.00,0.50,0.5,11.65,6.12,0.3,70.07,0.0
4,2.0,2021-03-01 00:02:13,2021-03-01 00:06:01,1.0,1.13,1.0,N,68,264,1.0,5.50,0.50,0.5,1.86,0.00,0.3,11.16,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925147,NaN,2021-03-29 12:39:29,2021-03-29 13:06:30,NaN,13.18,NaN,NaN,135,75,NaN,34.76,2.75,0.5,0.00,6.12,0.3,44.43,0.0
1925148,NaN,2021-03-29 12:11:00,2021-03-29 12:28:00,NaN,2.60,NaN,NaN,107,163,NaN,18.34,2.75,0.5,0.00,0.00,0.3,21.89,0.0
1925149,NaN,2021-03-29 12:50:00,2021-03-29 13:14:00,NaN,5.50,NaN,NaN,137,17,NaN,37.76,2.75,0.5,0.00,0.00,0.3,41.31,0.0
1925150,NaN,2021-03-29 12:12:00,2021-03-29 12:32:00,NaN,3.78,NaN,NaN,177,85,NaN,-1.55,2.75,0.5,0.00,23.50,0.3,25.50,0.0


In [31]:
df.shape

(1925152, 18)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925152 entries, 0 to 1925151
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 264.4+ MB


In [33]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [34]:
nulls = df.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,127920
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,127920
trip_distance,0
RatecodeID,127920
store_and_fwd_flag,127920
PULocationID,0
DOLocationID,0
payment_type,127920


In [35]:
df = df.dropna(how ='any')

In [36]:
nulls = df.isnull().sum().to_frame()# (to_frame():Convert Series to DataFrame),(sum():To sum the number of missing values),(isnull()to detect missing values)
nulls.columns = ["Missing values"] 
nulls

,Missing values
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
store_and_fwd_flag,0
PULocationID,0
DOLocationID,0
payment_type,0


In [37]:
df['RatecodeID'].value_counts()

1.0     1768883
2.0       19048
5.0        6123
4.0        1749
3.0        1378
99.0         39
6.0          12
Name: RatecodeID, dtype: int64

In [10]:
df = df[df.RatecodeID != 99.0]

In [11]:
df['RatecodeID'].value_counts()

1.0    1768883
2.0      19048
5.0       6123
4.0       1749
3.0       1378
6.0         12
Name: RatecodeID, dtype: int64

In [12]:
df['Day'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.day
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day
0,2.0,2021-03-01 00:22:02,2021-03-01 00:23:22,1.0,0.00,1.0,N,264,264,2.0,3.0,0.5,0.5,0.00,0.00,0.3,4.30,0.0,1
1,2.0,2021-03-01 00:24:48,2021-03-01 00:24:56,1.0,0.00,1.0,N,152,152,2.0,2.5,0.5,0.5,0.00,0.00,0.3,3.80,0.0,1
2,2.0,2021-03-01 00:25:17,2021-03-01 00:31:01,1.0,0.00,1.0,N,152,152,2.0,3.5,0.5,0.5,0.00,0.00,0.3,4.80,0.0,1
3,1.0,2021-03-01 00:07:40,2021-03-01 00:31:23,0.0,16.50,4.0,N,138,265,1.0,51.0,0.5,0.5,11.65,6.12,0.3,70.07,0.0,1
4,2.0,2021-03-01 00:02:13,2021-03-01 00:06:01,1.0,1.13,1.0,N,68,264,1.0,5.5,0.5,0.5,1.86,0.00,0.3,11.16,2.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797227,2.0,2021-03-31 23:01:03,2021-03-31 23:03:38,1.0,0.95,1.0,N,230,186,2.0,4.5,0.5,0.5,0.00,0.00,0.3,8.30,2.5,31
1797228,2.0,2021-03-31 23:05:19,2021-03-31 23:12:47,1.0,2.58,1.0,N,186,231,1.0,9.0,0.5,0.5,4.00,0.00,0.3,16.80,2.5,31
1797229,2.0,2021-03-31 23:15:42,2021-03-31 23:48:13,3.0,7.18,1.0,N,231,263,1.0,27.0,0.5,0.5,6.16,0.00,0.3,36.96,2.5,31
1797230,2.0,2021-03-31 23:50:08,2021-03-31 23:55:16,1.0,1.28,1.0,N,263,75,1.0,6.5,0.5,0.5,1.00,0.00,0.3,11.30,2.5,31


In [13]:
df1= df.drop(df[df.Day < 20 ].index)
df1

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Day
60,2.0,2021-02-28 23:57:22,2021-02-28 23:59:03,1.0,0.68,1.0,N,141,236,1.0,4.0,0.5,0.5,1.56,0.0,0.3,9.36,2.5,28
74,2.0,2021-02-28 23:59:53,2021-03-01 00:06:53,1.0,1.90,1.0,N,48,158,2.0,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,28
100,2.0,2021-02-28 23:56:41,2021-03-01 00:07:34,1.0,2.79,1.0,N,186,231,1.0,10.5,0.5,0.5,2.86,0.0,0.3,17.16,2.5,28
138,2.0,2021-02-28 23:57:53,2021-03-01 00:10:18,1.0,2.03,1.0,N,263,239,1.0,10.5,0.5,0.5,3.58,0.0,0.3,17.88,2.5,28
154,2.0,2021-02-28 23:57:06,2021-03-01 00:06:59,1.0,2.94,1.0,N,186,239,2.0,11.0,0.5,0.5,0.00,0.0,0.3,14.80,2.5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797227,2.0,2021-03-31 23:01:03,2021-03-31 23:03:38,1.0,0.95,1.0,N,230,186,2.0,4.5,0.5,0.5,0.00,0.0,0.3,8.30,2.5,31
1797228,2.0,2021-03-31 23:05:19,2021-03-31 23:12:47,1.0,2.58,1.0,N,186,231,1.0,9.0,0.5,0.5,4.00,0.0,0.3,16.80,2.5,31
1797229,2.0,2021-03-31 23:15:42,2021-03-31 23:48:13,3.0,7.18,1.0,N,231,263,1.0,27.0,0.5,0.5,6.16,0.0,0.3,36.96,2.5,31
1797230,2.0,2021-03-31 23:50:08,2021-03-31 23:55:16,1.0,1.28,1.0,N,263,75,1.0,6.5,0.5,0.5,1.00,0.0,0.3,11.30,2.5,31


In [14]:
df1.drop(['store_and_fwd_flag', 'PULocationID','DOLocationID','congestion_surcharge','tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1, inplace=True ) # delete two columns ,(axis=1:refers to column dimension)

In [15]:
df1.shape

(694888, 13)

In [16]:
df1_train , df1_test = train_test_split(df1 , test_size=.2, random_state=10)

In [17]:
df1_train, df1_val = train_test_split(df1_train, test_size=.25, random_state=5)

In [18]:
df1_train.shape

(416932, 13)

In [19]:
 df1_test.shape

(138978, 13)

In [20]:
df1_val.shape

(138978, 13)

In [21]:
###3-------------------------------------------

In [22]:
df1_train.VendorID = df1_train.VendorID.astype(str)

In [23]:
df1_train.RatecodeID = df1_train.RatecodeID.astype(str)

In [24]:
df1_train.payment_type = df1_train.payment_type.astype(str)

In [25]:
df1_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416932 entries, 1462865 to 1659755
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               416932 non-null  object 
 1   passenger_count        416932 non-null  float64
 2   trip_distance          416932 non-null  float64
 3   RatecodeID             416932 non-null  object 
 4   payment_type           416932 non-null  object 
 5   fare_amount            416932 non-null  float64
 6   extra                  416932 non-null  float64
 7   mta_tax                416932 non-null  float64
 8   tip_amount             416932 non-null  float64
 9   tolls_amount           416932 non-null  float64
 10  improvement_surcharge  416932 non-null  float64
 11  total_amount           416932 non-null  float64
 12  Day                    416932 non-null  int64  
dtypes: float64(9), int64(1), object(3)
memory usage: 44.5+ MB


In [26]:
df1_train['payment_type'].value_counts()

1.0    309224
2.0    103129
3.0      2793
4.0      1786
Name: payment_type, dtype: int64

In [27]:
df1_train['RatecodeID'].value_counts()

1.0    409143
2.0      5394
5.0      1582
4.0       421
3.0       391
6.0         1
Name: RatecodeID, dtype: int64

In [28]:
df_dummies_train = pd.get_dummies(data=df1_train)
df_dummies_train